# 测试考虑仅翻转逻辑超边对于解码的影响

在早期的工作中，我们从两个方面去思考：
1. 通常情况下，逻辑超边对应的错误率很小，即使加起来也不超过0.1。因此，我们可以简化为一个非常大的值，翻转为空，以及一个非常小的值，发生概率为0.0几。因此，通过收缩这个超边，本身不会影响最大概率的项。
2. 对于仅使得逻辑错误翻转的信息，无法提供syndrome与逻辑错误之间的对应关系。（但实际可能理解存在偏差，是提供全0syndrome与逻辑错误之间的关系）

为了验证上述想法，我们需要补充关于收缩所有超边的结果，同时从而计算得到概率分布。

首先，我们需要将代码进行对应的修改，包括Python和C++的代码，尽可能提供一个参数用于控制，从而能够进行实验比较二者的解码效果差异。

如果没有存在明确的解码效果差距，我们将存在近似的mld计算，默认不使用该部分超边。对于调用MLD进行计算的部分，默认使用超边仅逻辑错误翻转的超边。

## 测试代码运行是否正确

In [1]:
import stim
# X stabilizer to correct the Z error before circuit.
no_data_qubit_stim_file_X = "/home/normaluser/ck/noise2qec/data/surface_code/X/d3_r1/circuit_noisy_si1000_p10_no_stabilizer.stim"

surface_circuit = stim.Circuit.from_file(no_data_qubit_stim_file_X)


surface_circuit_dem = surface_circuit.detector_error_model(decompose_errors=False)

from hamld import HAMLD
from hamld.benchmark import LogicalErrorExpectationBenchmark

# contraction_code = normal, int, cpp-py and use_approx = True, False all same.
hamld_decoder = HAMLD(detector_error_model=surface_circuit_dem, order_method='greedy', slice_method='no_slice', contraction_code = "normal", use_approx=False)

benchmark = LogicalErrorExpectationBenchmark(
    decoder_function=hamld_decoder,
    d=3,
    r=1,
    p=10,
    noise_model="si1000",
    error_type="X",
)

logical = benchmark.run(have_stabilizer=False)[0]
print(logical)
print(benchmark.prob_dists)

if you have many shots. cudf can process data efficiently. pleased install cudf.
if you have many shots. cudf can process data efficiently. pleased install cudf.
0.0055708501515362954093
[[9.60375594e-01 2.77455712e-03]
 [6.49449753e-03 1.93008968e-05]
 [1.00276296e-02 3.12720034e-05]
 [6.84391880e-05 2.18895955e-07]
 [7.34426596e-03 1.17390803e-03]
 [4.97943311e-04 9.22947658e-06]
 [1.42155300e-03 1.61354585e-05]
 [1.42890219e-05 1.23655527e-07]
 [7.52488062e-03 1.49478912e-03]
 [5.08877911e-05 1.01126599e-05]
 [5.26820852e-04 1.69098774e-05]
 [3.56752720e-06 1.15624464e-07]
 [5.99407223e-05 2.04594909e-05]
 [3.91775405e-06 8.36052374e-07]
 [1.45781250e-05 2.85459850e-06]
 [3.44452368e-07 2.71874531e-08]]


In [2]:
import numpy as np

hamld_decoder.decode(np.array([False]*4))

(array([False]),
 {(False, False, False, False, False): 0.9603755944220893,
  (False, False, False, False, True): 0.0027745571178388775},
 0.9971192891228821)

## 测试逻辑超边到解码器的转换


In [3]:
# from hamld import HAMLDCpp_from_file
# from hamld.benchmark import generate_detector_error_model_path

# d = 3
# r = 1
# # real p is 10/10000
# p = 10
# noise_model = "si1000"
# error_type = "Z"
# related_path = "/home/normaluser/ck/noise2qec/data/surface_code"
# have_stabilizer = False

# approximatestrategy = "hyperedge_topk"
# approximate_param = 4
# priority = 0
# priority_topk = 4
# dem_file = generate_detector_error_model_path(d = d, r=r, p=p, noise_model=noise_model,
#                                               error_type=error_type, decomposed_error = False,
#                                               related_path=related_path, have_stabilizer = have_stabilizer)
# decoder =  HAMLDCpp_from_file(dem_path=str(dem_file), 
#                               approx_strategy=str(approximatestrategy), 
#                               approx_param=float(approximate_param), 
#                               priority=int(priority), 
#                               priority_topk = int(priority_topk),
#                               use_heuristic = False, alpha = float(0.05))


# benchmark = LogicalErrorExpectationBenchmark(
#     decoder_function=hamld_decoder,
#     d=3,
#     r=1,
#     p=10,
#     noise_model="si1000",
#     error_type="X",
# )

# logical = benchmark.run(have_stabilizer=False)[0]
# print(logical)
# print(benchmark.prob_dists)

这种情况并不特殊，我们可以直接将其视为超边，并在最后阶段统一进行收缩处理。
对于超图的选择，同样只需将其作为常规超边处理即可。

This scenario is not particularly exceptional; we can simply treat it as a hyperedge and perform contraction uniformly in the final stage.
Similarly, for hypergraph selection, it can also be handled as a conventional hyperedge.